In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [6]:
df = pd.read_csv("movies_metadata.csv")
df.head()

KeyboardInterrupt: 

In [5]:
df.value_counts()

adult  belongs_to_collection                                                                                                                                           budget     genres                                                                                                   homepage                                                                   id      imdb_id    original_language  original_title                                overview                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      